In [ ]:
!pip install langchain langchain-community langchain-chroma langchain-google-genai langchain-experimental langchain-text-splitters chromadb google-generativeai pillow numpy pandas sentence-transformers

## Setup and Imports

In [ ]:
import os
import base64
import io
import json
from typing import List, Dict, Any, Optional
import numpy as np
from PIL import Image
import google.generativeai as genai
import chromadb
from chromadb.utils import embedding_functions
from sentence_transformers import SentenceTransformer
import pandas as pd
from langchain.schema import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain_community.vectorstores import Chroma
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor
from langchain.chains import RetrievalQA, ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
from langchain.prompts import PromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

## Entire RAG configuration


In [ ]:
from dotenv import load_dotenv
import os

load_dotenv('/content/drive/MyDrive/api_keys.env')

hf_token = os.getenv("HF_TOKEN")
gemini_api = os.getenv("GEMINI_API_KEY")

class Config:
    """Configuration class for the RAG system"""

    def __init__(self):
        # API Configuration
        self.gemini_api_key = gemini_api

        # ChromaDB Configuration
        self.chroma_db_path = "./chroma_db"
        self.collection_name = "cat_health_multimodal"

        self.text_model = "gemini-1.5-pro"
        self.vision_model = "gemini-1.5-pro"
        self.embedding_model = "all-MiniLM-L6-v2"

        self.chunk_size = 1000
        self.chunk_overlap = 200
        self.max_retrieved_docs = 5
        self.temperature = 0.1

config = Config()

#Gemini Setup
genai.configure(api_key=config.gemini_api_key)
os.environ["GOOGLE_API_KEY"] = config.gemini_api_key
# Initialize models
text_model = genai.GenerativeModel(config.text_model)
vision_model = genai.GenerativeModel(config.vision_model)

# Initialize embedding model for text
embedding_model = SentenceTransformer(config.embedding_model)

## Langchain and ChromaDB Setup


In [ ]:
class LangChainMultimodalRAG:
    """LangChain-based multimodal RAG system for cat health analysis"""

    def __init__(self, config: Config):
        self.config = config

        # Initialize LangChain components
        self._setup_embeddings()
        self._setup_llm()
        self._setup_text_splitter()
        self._setup_vectorstore()
        self._setup_memory()
        self._setup_retrievers()
        self._setup_chains()

        self.vision_model = genai.GenerativeModel(config.vision_model)

    def _setup_embeddings(self):
        """Setup embedding models"""
        # Primary embeddings for text
        self.embeddings = SentenceTransformerEmbeddings(
            model_name=self.config.embedding_model
        )

    def _setup_llm(self):
        """Setup LangChain LLM"""
        self.llm = ChatGoogleGenerativeAI(
            model=self.config.text_model,
            temperature=self.config.temperature,
            convert_system_message_to_human=True
        )

    def _setup_text_splitter(self):
        """Setup document text splitter"""
        self.text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=self.config.chunk_size,
            chunk_overlap=self.config.chunk_overlap,
            separators=["\n\n", "\n", " ", ""]
        )

    def _setup_vectorstore(self):
        """Setup ChromaDB vectorstore with LangChain"""
        self.vectorstore = Chroma(
            collection_name=self.config.collection_name,
            embedding_function=self.embeddings,
            persist_directory=self.config.chroma_db_path
        )

    def _setup_memory(self):
        """Setup conversation memory"""
        self.memory = ConversationBufferMemory(
            memory_key="chat_history",
            return_messages=True,
            output_key="answer"
        )

    def _setup_retrievers(self):
        """Setup retrievers with different strategies"""

        # Basic retriever
        self.basic_retriever = self.vectorstore.as_retriever(
            search_type="similarity",
            search_kwargs={"k": self.config.max_retrieved_docs}
        )

        compressor = LLMChainExtractor.from_llm(self.llm)
        self.compression_retriever = ContextualCompressionRetriever(
            base_compressor=compressor,
            base_retriever=self.basic_retriever
        )

    def _setup_chains(self):
        """Setup LangChain chains"""
        self.cat_health_prompt = ChatPromptTemplate.from_messages([
            ("system", """You are a specialized veterinary assistant AI focused on cat health analysis.
            Use the provided context from the knowledge base to give comprehensive, helpful advice about potential cat health issues.

            IMPORTANT GUIDELINES:
            - Always recommend consulting with a veterinarian for proper diagnosis and treatment
            - Provide detailed analysis based on visible symptoms or described conditions
            - Suggest immediate care steps when appropriate
            - Explain when urgent veterinary care is needed
            - Consider both common and less common conditions
            - Ask clarifying questions if needed

            Context from knowledge base: {context}
            """),
            ("human", "{question}")
        ])

        # Create retrieval chain
        document_chain = create_stuff_documents_chain(self.llm, self.cat_health_prompt)
        self.retrieval_chain = create_retrieval_chain(self.compression_retriever, document_chain)

        # Conversational retrieval chain with memory
        self.conversational_chain = ConversationalRetrievalChain.from_llm(
            llm=self.llm,
            retriever=self.compression_retriever,
            memory=self.memory,
            return_source_documents=True,
            verbose=True
        )

    def add_documents_to_vectorstore(self, documents: List[Document]):
        """Add documents to the vectorstore"""
        # Split documents into chunks
        split_docs = self.text_splitter.split_documents(documents)

        # Add to vectorstore
        self.vectorstore.add_documents(split_docs)

        print(f"Added {len(split_docs)} document chunks to vectorstore")

    def analyze_cat_image(self, image_path: str, additional_context: str = "") -> Dict[str, Any]:
        """Analyze cat image using Gemini Vision"""
        try:
            # Load image
            image = Image.open(image_path)

            # Enhanced prompt for detailed veterinary analysis
            prompt = f"""
            You are an expert veterinarian analyzing this cat image. Please provide a comprehensive analysis including:

            1. PHYSICAL APPEARANCE:
               - Breed characteristics and physical features
               - Body condition score (thin, ideal, overweight)
               - Posture and positioning
               - Overall alertness and demeanor

            2. HEALTH INDICATORS:
               - Eyes: clarity, discharge, pupils, third eyelid visibility
               - Nose: discharge, color, breathing pattern
               - Mouth: visible teeth, gum color, drooling
               - Ears: position, discharge, scratching marks
               - Coat: shine, thickness, bald patches, parasites
               - Skin: color, lesions, swelling, scratches

            3. BEHAVIORAL OBSERVATIONS:
               - Activity level visible in image
               - Interaction with environment
               - Signs of comfort or distress
               - Hiding or seeking behavior

            4. POTENTIAL HEALTH CONCERNS:
               - List any visible abnormalities
               - Possible conditions based on visual cues
               - Urgency level (routine, prompt care needed, urgent)

            5. RECOMMENDATIONS:
               - Immediate care suggestions
               - When to contact veterinarian
               - Monitoring guidelines

            Additional context provided: {additional_context}

            Provide detailed, professional analysis while noting this is not a substitute for in-person veterinary examination.
            """

            response = self.vision_model.generate_content([prompt, image])

            return {
                "analysis": response.text,
                "image_path": image_path,
                "status": "success",
                "additional_context": additional_context
            }

        except Exception as e:
            return {
                "analysis": f"Error analyzing image: {str(e)}",
                "image_path": image_path,
                "status": "error",
                "additional_context": additional_context
            }

    def add_image_analysis_to_knowledge(self, image_path: str, additional_context: str = "",
                                      category: str = "image_analysis") -> str:
        """Analyze image and add to knowledge base"""
        analysis_result = self.analyze_cat_image(image_path, additional_context)

        if analysis_result["status"] == "success":
            # Create document for the analysis
            doc_content = f"""
            Image Analysis Report

            Image Path: {image_path}
            Additional Context: {additional_context}
            Analysis Date: {pd.Timestamp.now().isoformat()}

            Detailed Analysis:
            {analysis_result['analysis']}
            """
            doc = Document(
                page_content=doc_content,
                metadata={
                    "source": "image_analysis",
                    "category": category,
                    "image_path": image_path,
                    "timestamp": pd.Timestamp.now().isoformat(),
                    "type": "multimodal_analysis"
                }
            )

            # Add to vectorstore
            self.add_documents_to_vectorstore([doc])

            print(f"Image analysis added to knowledge base")
            return analysis_result["analysis"]
        else:
            print(f"Failed to analyze image: {analysis_result['analysis']}")
            return None

    def query_with_text_only(self, question: str) -> Dict[str, Any]:
        """Query using text only with retrieval chain"""

        try:
            result = self.retrieval_chain.invoke({"input": question})

            return {
                "answer": result["answer"],
                "source_documents": result.get("context", []),
                "question": question,
                "type": "text_only"
            }
        except Exception as e:
            return {
                "answer": f"Error processing query: {str(e)}",
                "source_documents": [],
                "question": question,
                "type": "error"
            }

    def query_with_image(self, question: str, image_path: str) -> Dict[str, Any]:
        """Query with both text and image analysis"""

        # First analyze the image
        image_analysis = self.analyze_cat_image(image_path)

        if image_analysis["status"] == "success":
            # Combine question with image analysis
            enhanced_question = f"""
            User Question: {question}

            Current Image Analysis:
            {image_analysis['analysis']}

            Based on both the question and the image analysis above, please provide a comprehensive response about potential health concerns and recommendations.
            """
            try:
                result = self.retrieval_chain.invoke({"input": enhanced_question})

                return {
                    "answer": result["answer"],
                    "source_documents": result.get("context", []),
                    "image_analysis": image_analysis["analysis"],
                    "question": question,
                    "image_path": image_path,
                    "type": "multimodal"
                }
            except Exception as e:
                return {
                    "answer": f"Error processing multimodal query: {str(e)}",
                    "source_documents": [],
                    "image_analysis": image_analysis["analysis"],
                    "question": question,
                    "type": "error"
                }
        else:
            print("Image analysis failed, falling back to text-only query")
            return self.query_with_text_only(question)

    def conversational_query(self, question: str) -> Dict[str, Any]:
        """Query with conversation memory"""

        try:
            result = self.conversational_chain.invoke({"question": question})

            return {
                "answer": result["answer"],
                "source_documents": result.get("source_documents", []),
                "chat_history": result.get("chat_history", []),
                "question": question,
                "type": "conversational"
            }
        except Exception as e:
            return {
                "answer": f"Error in conversational query: {str(e)}",
                "source_documents": [],
                "question": question,
                "type": "error"
            }

    def add_custom_knowledge(self, content: str, category: str = "custom",
                           source: str = "user_input") -> None:
        """Add custom knowledge to the system"""

        doc = Document(
            page_content=content,
            metadata={
                "source": source,
                "category": category,
                "timestamp": pd.Timestamp.now().isoformat(),
                "type": "custom_knowledge"
            }
        )

        self.add_documents_to_vectorstore([doc])
        print(f"Added custom knowledge to category: {category}")

    def get_vectorstore_stats(self) -> Dict[str, Any]:
        """Get statistics about the vectorstore"""
        try:
            collection = self.vectorstore._collection
            count = collection.count()

            return {
                "total_documents": count,
                "collection_name": self.config.collection_name,
                "embedding_model": self.config.embedding_model
            }
        except Exception as e:
            return {"error": f"Could not retrieve stats: {str(e)}"}

# Initialize the RAG system
rag_system = LangChainMultimodalRAG(config)

## Knowledge Base initialization and population


In [ ]:
def initialize_cat_health_knowledge():
    """Initialize the knowledge base with comprehensive cat health information"""

    cat_health_docs = [
        {
            "content": """
            Cat Eye Health and Common Problems

            Healthy cat eyes should be bright, clear, and free from excessive discharge. The pupils should respond to light changes, and the third eyelid (nictitating membrane) should not be visible.

            Common eye problems include:
            - Conjunctivitis: Inflammation causing redness, swelling, and discharge
            - Upper respiratory infections: Often cause watery eyes and sneezing
            - Corneal ulcers: Painful scratches on the eye surface, cause squinting
            - Glaucoma: Increased eye pressure, causes enlarged pupils and pain
            - Cataracts: Cloudiness in the lens, more common in older cats

            Warning signs requiring immediate veterinary care:
            - Sudden vision changes or blindness
            - Eye trauma or obvious pain
            - Green or yellow thick discharge
            - Pupils of different sizes
            - Cloudiness or color changes in the eye
            """,
            "category": "eye_health"
        },
        {
            "content": """
            Cat Respiratory Health and Breathing Issues

            Normal cat breathing is quiet and effortless, with 20-30 breaths per minute at rest.

            Common respiratory issues:
            - Upper respiratory infections (URI): Sneezing, nasal discharge, congestion
            - Asthma: Wheezing, coughing, open-mouth breathing
            - Pneumonia: Difficulty breathing, lethargy, loss of appetite
            - Foreign objects: Sudden onset breathing difficulty, pawing at face

            Emergency signs:
            - Open-mouth breathing at rest
            - Blue gums or tongue
            - Extreme difficulty breathing
            - Choking or gagging sounds
            - Collapse or weakness with breathing problems

            Preventive care includes keeping cats indoors, maintaining good ventilation, and avoiding smoke or strong chemicals.
            """,
            "category": "respiratory_health"
        },
        {
            "content": """
            Cat Skin and Coat Health Assessment

            A healthy cat coat is shiny, smooth, and free from bald patches, excessive dandruff, or parasites. Cats typically groom themselves 30-50% of their waking hours.

            Skin and coat problems:
            - Parasites: Fleas, mites, lice cause itching and hair loss
            - Allergies: Environmental or food allergies cause scratching and skin irritation
            - Bacterial infections: Hot spots, pustules, crusting lesions
            - Fungal infections: Ringworm causes circular bald patches
            - Hormonal issues: Symmetrical hair loss, often without itching
            - Stress-related: Over-grooming, bald spots from excessive licking

            Signs to watch for:
            - Excessive scratching or licking
            - Hair loss or thinning
            - Red, inflamed, or swollen skin
            - Unusual odors
            - Changes in grooming behavior
            """,
            "category": "skin_coat_health"
        },
        {
            "content": """
            Cat Pain Recognition and Behavioral Changes

            Cats are masters at hiding pain, making recognition challenging. Subtle behavioral changes are often the first indicators.

            Signs of pain or discomfort:
            - Changes in posture: Hunched back, reluctance to move
            - Altered mobility: Difficulty jumping, climbing, or using stairs
            - Behavioral changes: Hiding, aggression, withdrawn behavior
            - Eating changes: Loss of appetite or difficulty chewing
            - Litter box issues: Straining, avoiding the box, accidents
            - Vocalization changes: Increased crying, purring, or silence
            - Grooming changes: Over-grooming or neglecting grooming

            Chronic pain indicators:
            - Decreased activity and playfulness
            - Sleep pattern changes
            - Reduced interaction with family
            - Stiffness, especially after resting
            - Reluctance to be petted or handled
            """,
            "category": "pain_behavior"
        },
        {
            "content": """
            Cat Dental Health and Oral Care

            Dental disease affects over 70% of cats by age 3. Regular dental care prevents pain, tooth loss, and systemic health problems.

            Signs of dental problems:
            - Bad breath (halitosis)
            - Yellow or brown tartar buildup on teeth
            - Red, swollen, or bleeding gums
            - Difficulty eating or chewing
            - Pawing at the mouth or face
            - Drooling, especially if bloody
            - Loose or missing teeth
            - Weight loss due to eating difficulties

            Dental disease progression:
            1. Plaque accumulation
            2. Tartar formation
            3. Gingivitis (gum inflammation)
            4. Periodontitis (advanced gum disease)
            5. Tooth loss and bone damage

            Prevention includes dental treats, brushing (if tolerated), and regular veterinary dental cleanings.
            """,
            "category": "dental_health"
        },
        {
            "content": """
            Cat Digestive Health and Gastrointestinal Issues

            Normal cat digestion includes regular eating, drinking, and elimination patterns. Changes in these patterns often indicate health problems.

            Common digestive issues:
            - Hairballs: Normal but excessive vomiting may indicate problems
            - Gastritis: Stomach inflammation causing vomiting and loss of appetite
            - Inflammatory bowel disease: Chronic vomiting, diarrhea, weight loss
            - Constipation: Straining, hard stools, loss of appetite
            - Diarrhea: Loose stools, dehydration risk
            - Foreign body ingestion: Vomiting, lethargy, loss of appetite

            Warning signs:
            - Vomiting more than once per week
            - Blood in vomit or stool
            - Persistent diarrhea or constipation
            - Sudden loss of appetite lasting more than 24 hours
            - Abdominal pain or bloating
            - Significant weight loss
            """,
            "category": "digestive_health"
        },
        {
            "content": """
            Senior Cat Health and Age-Related Changes

            Cats are considered senior at 7-10 years old. Regular monitoring becomes crucial as age-related health issues develop.

            Common senior cat issues:
            - Arthritis: Joint stiffness, difficulty jumping, mobility issues
            - Kidney disease: Increased drinking/urinating, weight loss
            - Hyperthyroidism: Weight loss despite good appetite, hyperactivity
            - Diabetes: Increased drinking/urinating, weight loss, lethargy
            - Cognitive dysfunction: Disorientation, behavior changes
            - Vision/hearing loss: Bumping into objects, not responding to sounds
            - Dental disease: More severe with age, affects eating

            Senior care recommendations:
            - Bi-annual veterinary checkups
            - Blood work monitoring
            - Weight management
            - Environmental modifications for comfort
            - Pain management for arthritis
            - Mental stimulation to prevent cognitive decline
            """,
            "category": "senior_cat_health"
        }
    ]

    # Convert to LangChain documents
    documents = []
    for doc_data in cat_health_docs:
        doc = Document(
            page_content=doc_data["content"],
            metadata={
                "source": "veterinary_reference",
                "category": doc_data["category"],
                "timestamp": pd.Timestamp.now().isoformat(),
                "type": "reference_material"
            }
        )
        documents.append(doc)

    # Add to vectorstore
    rag_system.add_documents_to_vectorstore(documents)
    print(f"Initialized knowledge base with {len(documents)} comprehensive health documents")

# Initialize the knowledge base
initialize_cat_health_knowledge()

## Additional Utility Functions

In [ ]:
def smart_health_query(question: str, image_path: str = None, use_conversation: bool = False) -> str:
    """Enhanced query function with multiple modes"""

    print(f"\n{'='*50}")
    print(f"CAT HEALTH QUERY")
    print(f"{'='*50}")
    print(f"Question: {question}")
    if image_path:
        print(f"Image: {image_path}")
    print(f"Mode: {'Conversational' if use_conversation else 'Standard'}")
    print(f"{'='*50}\n")

    if use_conversation:
        # Use conversational chain with memory
        result = rag_system.conversational_query(question)
    elif image_path:
        # Multimodal query with image
        result = rag_system.query_with_image(question, image_path)
    else:
        # Text-only query
        result = rag_system.query_with_text_only(question)

    print("ANALYSIS RESULT:")
    print("-" * 30)
    print(result["answer"])

    if "source_documents" in result and result["source_documents"]:
        print(f"\nRetrieved {len(result['source_documents'])} relevant documents from knowledge base")

    if "image_analysis" in result:
        print(f"\nIMAGE ANALYSIS INCLUDED")

    return result["answer"]

def batch_analyze_cat_images(image_folder: str, file_pattern: str = "*") -> Dict[str, Any]:
    """Analyze multiple cat images and add to knowledge base"""
    import glob

    # Find image files
    extensions = ['jpg', 'jpeg', 'png', 'gif', 'bmp']
    image_files = []

    for ext in extensions:
        pattern = os.path.join(image_folder, f"{file_pattern}.{ext}")
        image_files.extend(glob.glob(pattern, recursive=True))
        image_files.extend(glob.glob(pattern.replace(ext, ext.upper()), recursive=True))

    results = {}
    successful_analyses = 0

    print(f"Found {len(image_files)} images to analyze")

    for i, image_path in enumerate(image_files, 1):
        print(f"Processing {i}/{len(image_files)}: {os.path.basename(image_path)}")

        # Add image analysis to knowledge base
        analysis = rag_system.add_image_analysis_to_knowledge(
            image_path,
            additional_context=f"Batch analysis {i}",
            category="batch_image_analysis"
        )

        if analysis:
            results[image_path] = {
                "status": "success",
                "analysis": analysis
            }
            successful_analyses += 1
        else:
            results[image_path] = {
                "status": "failed",
                "analysis": None
            }

    summary = {
        "total_images": len(image_files),
        "successful_analyses": successful_analyses,
        "failed_analyses": len(image_files) - successful_analyses,
        "results": results
    }

    print(f"\nBatch analysis complete: {successful_analyses}/{len(image_files)} successful")
    return summary

def add_personal_cat_profile(cat_name: str, breed: str, age: str,
                           medical_history: str, behavioral_notes: str):
    """Add comprehensive cat profile to knowledge base"""

    profile_content = f"""
    Personal Cat Profile: {cat_name}

    Basic Information:
    - Name: {cat_name}
    - Breed: {breed}
    - Age: {age}

    Medical History:
    {medical_history}

    Behavioral Patterns and Notes:
    {behavioral_notes}

    Profile Created: {pd.Timestamp.now().strftime('%Y-%m-%d %H:%M:%S')}

    This profile contains specific information about {cat_name}'s health patterns,
    behavioral tendencies, and medical history to provide personalized health analysis.
    """

    rag_system.add_custom_knowledge(
        content=profile_content,
        category=f"cat_profile_{cat_name.lower().replace(' ', '_')}",
        source="personal_profile"
    )

    print(f"Added comprehensive profile for {cat_name}")

def export_conversation_history():
    """Export conversation history and knowledge base summary"""

    # Get vectorstore stats
    stats = rag_system.get_vectorstore_stats()

    # Get conversation memory
    try:
        chat_history = rag_system.memory.chat_memory.messages
        formatted_history = []
        for msg in chat_history:
            formatted_history.append({
                "type": msg.__class__.__name__,
                "content": msg.content,
                "timestamp": pd.Timestamp.now().isoformat()
            })
    except:
        formatted_history = []

    export_data = {
        "export_timestamp": pd.Timestamp.now().isoformat(),
        "vectorstore_stats": stats,
        "conversation_history": formatted_history,
        "system_config": {
            "model": config.text_model,
            "embedding_model": config.embedding_model,
            "chunk_size": config.chunk_size,
            "max_retrieved_docs": config.max_retrieved_docs
        }
    }

    filename = f"cat_health_session_export_{pd.Timestamp.now().strftime('%Y%m%d_%H%M%S')}.json"
    with open(filename, 'w') as f:
        json.dump(export_data, f, indent=2)

    print(f"Session exported to: {filename}")
    return filename

## Some Demonstartion and Testing

In [ ]:
def run_comprehensive_demo():
    """Run a comprehensive demonstration of the system"""

    print("\n" + "="*60)
    print("LANGCHAIN MULTIMODAL CAT HEALTH RAG SYSTEM DEMO")
    print("="*60)

    # Show system stats
    stats = rag_system.get_vectorstore_stats()
    print(f"\nSYSTEM STATUS:")
    print(f"- Total documents in knowledge base: {stats.get('total_documents', 'Unknown')}")
    print(f"- Collection: {stats.get('collection_name', 'Unknown')}")
    print(f"- Embedding model: {stats.get('embedding_model', 'Unknown')}")

    # Demo 1: Basic health query
    print(f"\nDEMO 1: Basic Health Query")
    print("-" * 40)
    response1 = smart_health_query("My cat has been sneezing and has watery eyes for 2 days. What should I do?")

    # Demo 2: Add personal knowledge
    print(f"\nDEMO 2: Adding Personal Knowledge")
    print("-" * 40)
    rag_system.add_custom_knowledge(
        "My cat Whiskers always hides under the bed when she's not feeling well. She also stops eating her favorite treats when sick.",
        category="personal_behavior",
        source="owner_observation"
    )

    # Demo 3: Query with personal context
    print(f"\nDEMO 3: Query Using Personal Knowledge")
    print("-" * 40)
    response3 = smart_health_query("Whiskers is hiding under the bed and won't eat her treats. Is this concerning?")

    # Demo 4: Conversational query
    print(f"\nDEMO 4: Conversational Query with Memory")
    print("-" * 40)
    response4 = smart_health_query("What are the most common eye problems in cats?", use_conversation=True)
    response5 = smart_health_query("What about the treatment options for those conditions?", use_conversation=True)

    print(f"\nDemo tests were successful.The system is ready for use.")

    # Usage instructions
    print(f"\n" + "="*60)
    print("QUICK START GUIDE")
    print("="*60)
    print("1. Text queries: smart_health_query('What causes cat sneezing?')")
    print("2. Image analysis: smart_health_query('What do you see?', '/path/to/image.jpg')")
    print("3. Add cat profile: add_personal_cat_profile('Fluffy', 'Persian', '5 years', 'No major issues', 'Loves hiding in boxes')")
    print("4. Batch image analysis: batch_analyze_cat_images('/path/to/images/')")
    print("5. Conversational mode: smart_health_query('Question?', use_conversation=True)")

    print(f"\nSYSTEM FEATURES:")
    print("- LangChain-powered RAG architecture")
    print("- Multimodal analysis (text + images)")
    print("- Conversation memory")
    print("- Document chunking and compression")
    print("- ChromaDB vector storage")
    print("- Gemini Vision API integration")
    print("- Personal cat profile management")

    return stats

## Specialized Query functions for the image as well as text functionality

In [ ]:
def diagnose_with_image(image_path: str, symptoms: str = "", cat_profile: str = None) -> str:
    """Comprehensive diagnosis combining image and symptoms"""

    query = f"""
    COMPREHENSIVE HEALTH ASSESSMENT

    Visual Analysis: Please analyze the provided image for any health indicators.

    Reported Symptoms: {symptoms if symptoms else 'No additional symptoms reported'}

    Cat Profile Context: {cat_profile if cat_profile else 'No specific profile provided'}

    Please provide:
    1. Visual assessment findings
    2. Correlation with reported symptoms
    3. Possible conditions/diagnoses
    4. Recommended next steps
    5. Monitoring guidelines
    """

    return smart_health_query(query, image_path)

def preventive_care_plan(cat_name: str, age: str, breed: str, lifestyle: str) -> str:
    """Generate preventive care recommendations"""

    query = f"""
    PREVENTIVE CARE PLAN REQUEST

    Cat Details:
    - Name: {cat_name}
    - Age: {age}
    - Breed: {breed}
    - Lifestyle: {lifestyle}

    Please create a comprehensive preventive care plan including:
    1. Age-appropriate health screenings
    2. Vaccination schedule recommendations
    3. Breed-specific health monitoring
    4. Lifestyle-related preventive measures
    5. Nutrition and exercise recommendations
    6. Environmental enrichment suggestions
    7. Warning signs to monitor for this specific cat
    """

    response = smart_health_query(query)

    rag_system.add_custom_knowledge(
        content=f"Preventive care plan for {cat_name}: {response}",
        category=f"preventive_care_{cat_name.lower().replace(' ', '_')}",
        source="preventive_plan"
    )

    return response

def medication_interaction_check(medications: List[str], condition: str) -> str:
    """Check for medication interactions and contraindications"""

    query = f"""
    MEDICATION INTERACTION AND SAFETY CHECK

    Current/Proposed Medications: {', '.join(medications)}
    Condition Being Treated: {condition}

    Please analyze:
    1. Potential drug interactions between listed medications
    2. Contraindications for the specific condition
    3. Common side effects to monitor
    4. Administration guidelines
    5. When to contact veterinarian
    6. Signs of adverse reactions

    IMPORTANT: This is for informational purposes only. Always consult with a veterinarian before starting, stopping, or changing medications.
    """


## Reporting and Analytics

In [ ]:
def generate_health_report(cat_name: str, time_period: str = "recent") -> str:
    """Generate comprehensive health report for a cat"""

    query = f"""
    COMPREHENSIVE HEALTH REPORT REQUEST

    Cat Name: {cat_name}
    Time Period: {time_period}

    Please compile a comprehensive health report including:

    1. HEALTH STATUS SUMMARY
       - Overall health assessment based on available data
       - Key findings from recent observations/analyses

    2. IDENTIFIED CONCERNS
       - Any health issues noted
       - Severity and urgency assessment

    3. BEHAVIORAL PATTERNS
       - Notable behavioral observations
       - Changes in patterns over time

    4. PREVENTIVE CARE STATUS
       - Vaccinations and health maintenance
       - Screening recommendations

    5. RECOMMENDATIONS
       - Immediate actions needed
       - Long-term health management
       - Follow-up requirements

    6. MONITORING PLAN
       - What to watch for
       - When to reassess

    Please base this report on all available information about {cat_name} in the knowledge base.
    """

    report = smart_health_query(query)

    # Save report to knowledge base
    rag_system.add_custom_knowledge(
        content=f"Health Report for {cat_name} ({pd.Timestamp.now().strftime('%Y-%m-%d')}): {report}",
        category=f"health_report_{cat_name.lower().replace(' ', '_')}",
        source="health_report"
    )

    return report

def analyze_knowledge_gaps() -> Dict[str, Any]:
    """Analyze what information might be missing from the knowledge base"""

    query = """
    KNOWLEDGE BASE ANALYSIS

    Based on the available information in the knowledge base, please identify:

    1. Common cat health topics that may need more comprehensive coverage
    2. Gaps in breed-specific information
    3. Age-group specific information that could be expanded
    4. Emergency care protocols that might be missing
    5. Preventive care information gaps
    6. Behavioral health topics needing more coverage

    Suggest specific areas where additional veterinary knowledge would improve the system's capabilities.
    """

    result = rag_system.query_with_text_only(query)

    return {
        "analysis": result["answer"],
        "recommendations": "Consider adding more information in identified gap areas",
        "knowledge_base_stats": rag_system.get_vectorstore_stats()
    }

## Final Execution and Usage

In [ ]:
if __name__ == "__main__":
    # Execute the demo
    demo_stats = run_comprehensive_demo()

    print(f"\n" + "="*60)
    print("ADVANCED FEATURES AVAILABLE")
    print("="*60)
    print("• Emergency Assessment: advanced_analyzer.emergency_assessment('symptoms', 'image.jpg')")
    print("• Symptom Tracking: advanced_analyzer.symptom_progression_tracker('CatName', symptom_list)")
    print("• Comparative Analysis: advanced_analyzer.comparative_analysis('question', ['cat1', 'cat2'])")
    print("• Diagnosis with Image: diagnose_with_image('image.jpg', 'symptoms', 'profile')")
    print("• Preventive Care Plan: preventive_care_plan('Name', 'Age', 'Breed', 'Lifestyle')")
    print("• Medication Check: medication_interaction_check(['med1', 'med2'], 'condition')")
    print("• Health Reports: generate_health_report('CatName', 'recent')")
    print("• Knowledge Gap Analysis: analyze_knowledge_gaps()")

    print(f"\nREADY TO USE!")
    print("Replace 'YOUR_GEMINI_API_KEY_HERE' with your actual API key and start analyzing!")

    print(f"\nRemember to add your cat's personal information using:")
    print("add_personal_cat_profile('YourCatName', 'Breed', 'Age', 'Medical History', 'Behavioral Notes')")

## Some example usage functions

In [ ]:
def example_scenarios():
    """Show practical usage scenarios"""

    print("\n" + "="*60)
    print("PRACTICAL USAGE SCENARIOS")
    print("="*60)

    scenarios = [
        {
            "title": "Scenario 1: New Symptom Investigation",
            "code": """
# Your cat shows new symptoms
symptoms = "Not eating, hiding, seems lethargic for 2 days"
assessment = advanced_analyzer.emergency_assessment(symptoms)
print(f"Urgency Level: {assessment['urgency_level']}")
print(assessment['full_assessment'])
            """
        },
        {
            "title": "Scenario 2: Photo Analysis with Context",
            "code": """
# You notice something in a photo
image_path = "/path/to/cat_photo.jpg"
context = "Cat seems to be squinting and pawing at left eye"
diagnosis = diagnose_with_image(image_path, context, "Fluffy - 3yr old Persian")
print(diagnosis)
            """
        },
        {
            "title": "Scenario 3: Tracking Symptoms Over Time",
            "code": """
# Track symptoms across multiple days
symptom_entries = [
    {"date": "2024-01-15", "symptoms": "Mild sneezing", "severity": "mild"},
    {"date": "2024-01-16", "symptoms": "Sneezing + watery eyes", "severity": "moderate"},
    {"date": "2024-01-17", "symptoms": "Same + reduced appetite", "severity": "concerning"}
]
advanced_analyzer.symptom_progression_tracker("Whiskers", symptom_entries)
            """
        },
        {
            "title": "Scenario 4: Preventive Care Planning",
            "code": """
# Create preventive care plan
care_plan = preventive_care_plan(
    cat_name="Luna",
    age="7 years (senior)",
    breed="Maine Coon",
    lifestyle="Indoor only, lives with other cats"
)
print(care_plan)
            """
        },
        {
            "title": "Scenario 5: Multi-cat Household Analysis",
            "code": """
# Compare how a condition might affect different cats
analysis = advanced_analyzer.comparative_analysis(
    "How would upper respiratory infection affect cats differently?",
    ["Luna (Maine Coon, 7yr)", "Whiskers (Persian, 3yr)", "Shadow (DSH, 1yr)"]
)
print(analysis)
            """
        }
    ]

    for scenario in scenarios:
        print(f"\n{scenario['title']}")
        print("-" * len(scenario['title']))
        print(scenario['code'])

    print(f"\nPro Tips:")
    print("- Always provide context about your specific cat")
    print("- Use high-quality, well-lit photos for best image analysis")
    print("- Track symptoms over time for pattern recognition")
    print("- Combine multiple data sources (photos, behavior, symptoms)")
    print("- Regular preventive care planning helps catch issues early")

# Show example scenarios
example_scenarios()

print(f"\nLangChain Multimodal Cat Health RAG System Complete!")
print(f"This system provides enterprise-grade RAG architecture with:")
print(f"• Advanced retrieval and compression")
print(f"• Conversation memory and context tracking")
print(f"• Multimodal analysis capabilities")
print(f"• Extensible knowledge base management")
print(f"• Professional veterinary-focused prompting")
print(f"\nYour cat's health is now powered by cutting-edge AI!")